In [ ]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

LangChain으로 구축한 애플리케이션은 여러 단계에 걸쳐 LLM 호출을 여러 번 사용하게 됩니다. 이러한 애플리케이션이 점점 더 복잡해짐에 따라, 체인이나 에이전트 내부에서 정확히 무슨 일이 일어나고 있는지 조사할 수 있는 능력이 매우 중요해집니다. 이를 위한 최선의 방법은 LangSmith를 사용하는 것입니다.

In [2]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("hhw_langchain_toy_project")

LangSmith 추적을 시작합니다.
[프로젝트명]
hhw_langchain_toy_project


RAG 기본 파이프라인(1~8단계)

In [12]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.chat_models import ChatOllama

# Ollama 모델을 불러옵니다.
llm = ChatOllama(model="gemma3:1b")

ollama_embeddings = OllamaEmbeddings(
    model="nomic-embed-text",
    # model="chatfire/bge-m3:q8_0" # BGE-M3
)


In [5]:
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.6/18.6 MB 9.1 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


아래는 기본적인 RAG 구조 이해를 위한 뼈대코드(skeleton code) 입니다.

각 단계별 모듈의 내용을 앞으로 상황에 맞게 변경하면서 문서에 적합한 구조를 찾아갈 수 있습니다.

(각 단계별로 다양한 옵션을 설정하거나 새로운 기법을 적용할 수 있습니다.)

In [6]:
# 단계 1: 문서 로드(Load Documents)
loader = PyMuPDFLoader("data/SPRI_AI_Brief_2023년12월호_F.pdf")
docs = loader.load()
print(f"문서의 페이지수: {len(docs)}")


문서의 페이지수: 23


In [7]:
# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)
print(f"분할된 청크의수: {len(split_documents)}")


분할된 청크의수: 43


In [9]:
# 단계 3: 임베딩(Embedding) 생성
embeddings = ollama_embeddings

In [13]:
# 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 생성합니다.
vectorstore = Chroma.from_documents(
    documents=split_documents, embedding=ollama_embeddings, collection_name="my_db"
)

In [14]:
# 단계 5: 검색기(Retriever) 생성
# 문서에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

In [15]:
# 검색기에 쿼리를 날려 검색된 chunk 결과를 확인합니다.
retriever.invoke("삼성전자가 자체 개발한 AI 의 이름은?")

[Document(id='fa942ed3-f582-4043-880d-1d8f27bfbc3f', metadata={'author': 'dj', 'creationDate': "D:20231208132838+09'00'", 'creationdate': '2023-12-08T13:28:38+09:00', 'creator': 'Hwp 2018 10.0.0.13462', 'file_path': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'format': 'PDF 1.4', 'keywords': '', 'modDate': "D:20231208132838+09'00'", 'moddate': '2023-12-08T13:28:38+09:00', 'page': 3, 'producer': 'Hancom PDF 1.3.0.542', 'source': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'subject': '', 'title': '', 'total_pages': 23, 'trapped': ''}, page_content='1. 정책/법제  \n2. 기업/산업 \n3. 기술/연구 \n 4. 인력/교육\n미국, 안전하고 신뢰할 수 있는 AI 개발과 사용에 관한 행정명령 발표 \nn 미국 바이든 대통령이 ‘안전하고 신뢰할 수 있는 AI 개발과 사용에 관한 행정명령’에 서명하고 \n광범위한 행정 조치를 명시\nn 행정명령은 △AI의 안전과 보안 기준 마련 △개인정보보호 △형평성과 시민권 향상 △소비자 \n보호 △노동자 지원 △혁신과 경쟁 촉진 △국제협력을 골자로 함\nKEY Contents\n£ 바이든 대통령, AI 행정명령 통해 안전하고 신뢰할 수 있는 AI 개발과 활용 추진\nn 미국 바이든 대통령이 2023년 10월 30일 연방정부 차원에서 안전하고 신뢰할 수 있는 AI 개발과 \n사용을 보장하기 위한 행정명령을 발표\n∙행정명령은 △AI의 안전과 보안 기준 마련 △개인정보보호 △형평성과 시민권 향상 △소비자 보호 \n△노동자 지

In [16]:
# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Question: 
{question} 
#Context: 
{context} 

#Answer:"""
)

In [17]:
# 단계 7: 언어모델(LLM) 생성
# 모델(LLM) 을 생성합니다.
llm = ChatOllama(model="gemma3:1b")

In [18]:
# 단계 8: 체인(Chain) 생성
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [19]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "삼성전자가 자체 개발한 AI 의 이름은?"
response = chain.invoke(question)
print(response)

Okay, here's a breakdown of the provided text, summarizing the key points and focusing on the most important information:

**Summary of the Text:**

The text is a press release from the G7 (Group of Seven nations) announcing a new international code of conduct for advanced AI systems.  Here's a breakdown of the key takeaways:

* **Context:** The G7 is addressing the increasing risks associated with advanced AI systems.
* **Initiative:** They've established a "Hiroshima Process" (a voluntary framework) to encourage AI companies to proactively assess and mitigate risks.
* **The Code of Conduct:** This process involves a series of steps:
    * **Risk Assessment:**  Evaluating potential risks throughout the AI lifecycle.
    * **Transparency & Accountability:**  Making information about AI systems' capabilities and limitations publicly available.
    * **Security & Control:**  Ensuring the safety and control of AI systems.
* **Hiroshima Process Details:** The process involves a series of m